In [1]:
!gdown --id 104TU-gdBAtalfRyaauuKaOobhGbB52Zf

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=104TU-gdBAtalfRyaauuKaOobhGbB52Zf
To: /content/data.zip
100% 649k/649k [00:00<00:00, 146MB/s]


In [3]:
!unzip data.zip

Archive:  data.zip
replace simple-dstc2-trn.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: simple-dstc2-trn.json   
replace simple-dstc2-tst.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace simple-dstc2-tst.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: simple-dstc2-tst.json   
replace simple-dstc2-val.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: simple-dstc2-val.json   
replace slot_vals.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: slot_vals.json          
replace ner_intent.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ner_intent.json         
replace simple-dstc2-templates.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: yy
  inflating: simple-dstc2-templates.txt  
replace ner_intent_test.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ner_intent_test.json    
replace ner_intent_train.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace ner_

In [4]:
!pip install transformers

y
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import json
from transformers import BertTokenizer, TFBertModel
import numpy as np

In [52]:
def read_file(path):
  f = open(path, 'r')
  data = json.loads(f.read())
  sentences,nertags,intents = ([],[],[])
  for key,value in data.items():
    sentences.append(key)
    nertags.append(' '.join(data[key]['NERTAGS']))
    intents.append(data[key]['INTENTS'])
  f.close()
  return sentences, nertags,intents

In [53]:
train_sentence,train_nertag, train_intent = read_file('ner_intent_train.json')
test_sentence, test_nertag, test_intent = read_file('ner_intent_test.json')
valid_sentence, valid_nertag, valid_intent = read_file('ner_intent_valid.json')

In [8]:
print(train_sentence[200])
print(train_nertag[200])
print(train_intent[200])

im looking for an expensive restaurant in the north part of town
O O O O B-pricerange O O O B-area O O O
inform_pricerange inform_area


In [9]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [10]:
def encode_text(tokenizer, text_sequences, max_len):
  input_ids=[]
  attention_masks=[]

  for sent in text_sequences:
    bert_inp = tokenizer.encode_plus(sent, add_special_tokens = True, max_length=max_len, pad_to_max_length = True, 
                                        return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

  input_ids = np.asarray(input_ids)
  attention_masks = np.array(attention_masks)

  return input_ids, attention_masks

In [11]:
max_length = 30
train_inp, train_mask = encode_text(bert_tokenizer, train_sentence, max_length)
test_inp, test_mask = encode_text(bert_tokenizer, test_sentence, max_length)
valid_inp, valid_mask = encode_text(bert_tokenizer, valid_sentence, max_length)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [12]:
print(train_sentence[1])
print(train_inp[1])
print(bert_tokenizer.convert_ids_to_tokens(train_inp[1]))
print(train_mask[1])

any
[ 101 2151  102    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
['[CLS]', 'any', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [14]:
seq_out_tokenizer = Tokenizer(filters='', oov_token="UNK", lower=False)
seq_out_tokenizer.fit_on_texts(train_nertag)


In [15]:
seq_out_tokenizer.index_word[0] = 'PAD'
seq_out_tokenizer.word_index['PAD'] = 0

In [16]:
seq_out_tokenizer.word_index

{'UNK': 1,
 'O': 2,
 'B-food': 3,
 'B-area': 4,
 'B-pricerange': 5,
 'I-food': 6,
 'I-pricerange': 7,
 'B-this': 8,
 'I-this': 9,
 'I-area': 10,
 'B-name': 11,
 'I-name': 12,
 'PAD': 0}

In [17]:
seq_out_tokenizer.index_word

{1: 'UNK',
 2: 'O',
 3: 'B-food',
 4: 'B-area',
 5: 'B-pricerange',
 6: 'I-food',
 7: 'I-pricerange',
 8: 'B-this',
 9: 'I-this',
 10: 'I-area',
 11: 'B-name',
 12: 'I-name',
 0: 'PAD'}

In [18]:
train_slots_tokenized = seq_out_tokenizer.texts_to_sequences(train_nertag)
test_slots_tokenized = seq_out_tokenizer.texts_to_sequences(test_nertag)
valid_slots_tokenized = seq_out_tokenizer.texts_to_sequences(valid_nertag)

In [19]:
def padding_slots(tokenized_slots, max_len):
  final = np.zeros(shape=(len(tokenized_slots),max_len), dtype='int32')
  for i,slot in enumerate(tokenized_slots):
    final[i, 1:len(slot)+1] = slot
  return final

In [20]:
train_slots = padding_slots(train_slots_tokenized, max_length)
valid_slots = padding_slots(valid_slots_tokenized, max_length)
test_slots = padding_slots(test_slots_tokenized, max_length)

In [21]:
train_slots

array([[0, 5, 2, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0],
       [0, 4, 0, ..., 0, 0, 0],
       ...,
       [0, 5, 2, ..., 0, 0, 0],
       [0, 4, 0, ..., 0, 0, 0],
       [0, 2, 3, ..., 0, 0, 0]], dtype=int32)

In [22]:
print(train_slots[200])
print(train_slots[0])

[0 2 2 2 2 5 2 2 2 4 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 5 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## Q1


### a

In [23]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, f1_score

In [24]:
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super(MetricsCallback, self).__init__()
    def  on_train_begin(self,logs={}):
      self.f1_macro=[]
    def on_epoch_end(self, epoch, logs=None):
      train_preds = self.model.predict([train_inp, train_mask])
      pred = train_preds.reshape(train_preds.shape[0]*train_preds.shape[1], train_preds.shape[2])
      y_pred = [np.argmax(elem) for elem in pred]
      y_true = train_slots.reshape(train_slots.shape[0]*train_slots.shape[1])
      score=f1_score(y_true, y_pred, average='macro')
      self.f1_macro.append(score)
      print(" F1 macro :",score)

In [25]:
metrics=MetricsCallback()

In [26]:
slot_num = len(seq_out_tokenizer.word_index)

In [27]:
bert = TFBertModel.from_pretrained('bert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(max_length,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(max_length,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[0]
y = tf.keras.layers.Dense(slot_num, activation='softmax', name='outputs')(embeddings)

slotfilling_model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

slotfilling_model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

slotfilling_model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 30)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 30,                                            

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
cp_callback = ModelCheckpoint('/content/drive/MyDrive/slotfilling-{epoch:03d}--{val_accuracy:03f}.h5', 
                             verbose=1, monitor='val_accuracy',save_best_only=True, 
                             save_weights_only=True, mode='max') 

In [32]:
history = slotfilling_model.fit([train_inp, train_mask], train_slots,
                       batch_size = 32,
                       epochs = 2,
                       validation_data = ([valid_inp, valid_mask], valid_slots), callbacks=[cp_callback, metrics])

Epoch 1/2
61/61 [==============================] - ETA: 0s - loss: 0.0466 - accuracy: 0.9883
Epoch 1: val_accuracy improved from -inf to 0.99028, saving model to /content/drive/MyDrive/slotfilling-001--0.990282.h5
61/61 [==============================] - 5s 28ms/step
 F1 macro : 0.553846922610249
61/61 [==============================] - 11s 185ms/step - loss: 0.0466 - accuracy: 0.9883 - val_loss: 0.0330 - val_accuracy: 0.9903
Epoch 2/2
61/61 [==============================] - ETA: 0s - loss: 0.0322 - accuracy: 0.9918
Epoch 2: val_accuracy improved from 0.99028 to 0.99347, saving model to /content/drive/MyDrive/slotfilling-002--0.993467.h5
61/61 [==============================] - 2s 29ms/step
 F1 macro : 0.7005437577000233
61/61 [==============================] - 8s 129ms/step - loss: 0.0322 - accuracy: 0.9918 - val_loss: 0.0239 - val_accuracy: 0.9935


In [33]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
!gdown --id 1-m4ATc2ZvrDnEGVgVXpElAsmD0zy-f2k

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-m4ATc2ZvrDnEGVgVXpElAsmD0zy-f2k
To: /content/slotfilling-002--0.993467.h5
100% 438M/438M [00:01<00:00, 272MB/s]


In [35]:
slotfilling_model.load_weights('slotfilling-002--0.989381.h5')

In [36]:
slotfilling_model.evaluate([test_inp, test_mask], test_slots)

44/44 [==============================] - 1s 28ms/step - loss: 0.0373 - accuracy: 0.9901


[0.03727363795042038, 0.9901394844055176]

### b

In [37]:
!gdown --id 1-ay5mBPEFuzHGZpDnBJIA5e_ONAIcvjW

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-ay5mBPEFuzHGZpDnBJIA5e_ONAIcvjW
To: /content/slot-test.txt
100% 587/587 [00:00<00:00, 1.09MB/s]


In [38]:
def fill_slots(sentence):
  slots_dict = {}
  sent_inp, sent_mask = encode_text(bert_tokenizer, [sentence], max_length)
  predicted_slots = slotfilling_model.predict([sent_inp, sent_mask])
  slots = np.argmax(predicted_slots, axis=-1)
  slots = [seq_out_tokenizer.index_word[w_idx] for w_idx in slots[0]]
  sent_len = len(sentence.split())+1
  ner_tags = slots[1:sent_len]
  indices = [i for i in range(len(ner_tags)) if ner_tags[i] != 'O']
  for i in indices:
    slot = ner_tags[i].split('-')[1]
    slots_dict[slot] = sentence.split()[i]
  return slots_dict, ner_tags

In [39]:
with open('slot-test.txt') as f:
  for sent in f:
    sent = sent.strip()
    slots, ner_tags = fill_slots(sent)
    print(sent)
    print(ner_tags)
    print(slots)
    print('\n\n')

1/1 [==============================] - 0s 50ms/step
cheap restaurant
['B-pricerange', 'O']
{'pricerange': 'cheap'}



1/1 [==============================] - 0s 45ms/step


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


im looking for a cheap restaurant in the south part of town
['O', 'O', 'O', 'O', 'B-pricerange', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O']
{'pricerange': 'cheap', 'area': 'south'}



1/1 [==============================] - 0s 44ms/step
i want a restaurant in the north part of town
['O', 'O', 'O', 'O', 'O', 'O', 'B-area', 'O', 'O', 'O']
{'area': 'north'}



1/1 [==============================] - 0s 44ms/step
what is the address and phone number
['O', 'O', 'O', 'O', 'O', 'O', 'O']
{}



1/1 [==============================] - 0s 45ms/step
i got to find a cheap restaurant that serves mexican food
['O', 'O', 'O', 'O', 'O', 'B-pricerange', 'O', 'O', 'O', 'B-food', 'O']
{'pricerange': 'cheap', 'food': 'mexican'}



1/1 [==============================] - 0s 46ms/step
okay and uh whats their phone number please
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
{}



1/1 [==============================] - 0s 45ms/step
i would like to find a moderately priced restaurant in the north part of town
['O', 'O', '

##Q2 

### a

In [54]:
from sklearn.preprocessing import MultiLabelBinarizer

In [55]:
train_intent

[['inform_pricerange'],
 ['inform_this'],
 ['inform_area'],
 ['request_addr'],
 ['request_phone'],
 ['thankyou', 'bye'],
 ['inform_food'],
 ['inform_food'],
 ['request_addr'],
 ['unknown'],
 ['request_phone'],
 ['inform_pricerange', 'inform_area'],
 ['inform_area'],
 ['inform_this'],
 ['inform_food'],
 ['inform_food'],
 ['request_addr', 'request_phone'],
 ['inform_food', 'inform_pricerange'],
 ['confirm_pricerange'],
 ['confirm_food', 'confirm_pricerange'],
 ['request_phone'],
 ['negate', 'bye'],
 ['inform_food', 'inform_pricerange'],
 ['affirm'],
 ['reqalts', 'inform_food'],
 ['request_area'],
 ['request_phone'],
 ['request_area'],
 ['thankyou', 'bye'],
 ['inform_pricerange', 'inform_area'],
 ['negate', 'reqalts'],
 ['request_phone'],
 ['bye'],
 ['inform_food'],
 ['inform_area'],
 ['inform_pricerange', 'inform_area'],
 ['inform_this'],
 ['request_pricerange'],
 ['request_phone', 'request_postcode'],
 ['inform_food'],
 ['inform_area'],
 ['inform_food'],
 ['inform_area'],
 ['inform_area

In [57]:
intent_list = []
for x in train_intent:
  for y in x:
    if y not in intent_list:
      intent_list.append(y)

for x in valid_intent:
  for y in x:
    if y not in intent_list:
      intent_list.append(y)

for x in test_intent:
  for y in x:
    if y not in intent_list:
      intent_list.append(y)

In [58]:
mlb = MultiLabelBinarizer(classes=intent_list)

In [59]:
train_labels = mlb.fit_transform(train_intent)
valid_labels = mlb.fit_transform(valid_intent)
test_labels = mlb.fit_transform(test_intent)

In [60]:
intent_num = len(intent_list)

In [61]:
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super(MetricsCallback, self).__init__()
    def  on_train_begin(self,logs={}):
      self.f1_macro=[]
    def on_epoch_end(self, epoch, logs=None):
      y_pred = intent_model.predict([train_inp, train_mask])
      y_pred[y_pred>=0.5] = 1
      y_pred[y_pred<0.5] = 0
      score=f1_score(train_labels, y_pred, average='macro')
      self.f1_macro.append(score)
      print(" F1 macro :",score)

In [62]:
metrics=MetricsCallback()

In [63]:
bert = TFBertModel.from_pretrained('bert-base-uncased')

input_ids = tf.keras.layers.Input(shape=(max_length,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(max_length,), name='attention_mask', dtype='int32')

embeddings = bert(input_ids, attention_mask=mask)[1]
x = tf.keras.layers.Dropout(0.2)(embeddings)
y = tf.keras.layers.Dense(intent_num, activation='sigmoid', name='outputs')(x)

intent_model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

intent_model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

intent_model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 30)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 30)]         0           []                               
                                                                                                  
 tf_bert_model_2 (TFBertModel)  TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 30,                                          

In [64]:
cp_callback = ModelCheckpoint('/content/drive/MyDrive/intent-detection-{epoch:03d}--{val_accuracy:03f}.h5', 
                             verbose=1, monitor='val_accuracy',save_best_only=True, 
                             save_weights_only=True, mode='max') 

In [65]:
history = intent_model.fit([train_inp, train_mask], train_labels,
                       batch_size = 32,
                       epochs = 20,
                       validation_data = ([valid_inp, valid_mask], valid_labels), callbacks=[cp_callback, metrics])

Epoch 1/20
61/61 [==============================] - ETA: 0s - loss: 0.3708 - accuracy: 0.1873
Epoch 1: val_accuracy improved from -inf to 0.32756, saving model to /content/drive/MyDrive/intent-detection-001--0.327562.h5
61/61 [==============================] - 5s 27ms/step
 F1 macro : 0.00865440552142087
61/61 [==============================] - 30s 223ms/step - loss: 0.3708 - accuracy: 0.1873 - val_loss: 0.1523 - val_accuracy: 0.3276
Epoch 2/20
 3/61 [>.............................] - ETA: 3s - loss: 0.1541 - accuracy: 0.3542

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


61/61 [==============================] - ETA: 0s - loss: 0.1341 - accuracy: 0.4190
Epoch 2: val_accuracy improved from 0.32756 to 0.52701, saving model to /content/drive/MyDrive/intent-detection-002--0.527008.h5
61/61 [==============================] - 2s 29ms/step
 F1 macro : 0.14009761063260992
61/61 [==============================] - 8s 130ms/step - loss: 0.1341 - accuracy: 0.4190 - val_loss: 0.1010 - val_accuracy: 0.5270
Epoch 3/20
 3/61 [>.............................] - ETA: 3s - loss: 0.1053 - accuracy: 0.5208

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


61/61 [==============================] - ETA: 0s - loss: 0.0929 - accuracy: 0.5727
Epoch 3: val_accuracy improved from 0.52701 to 0.62535, saving model to /content/drive/MyDrive/intent-detection-003--0.625346.h5
61/61 [==============================] - 2s 27ms/step
 F1 macro : 0.2118694174460284
61/61 [==============================] - 8s 126ms/step - loss: 0.0929 - accuracy: 0.5727 - val_loss: 0.0724 - val_accuracy: 0.6253
Epoch 4/20
 3/61 [>.............................] - ETA: 3s - loss: 0.0753 - accuracy: 0.6562

...

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


61/61 [==============================] - ETA: 0s - loss: 0.0080 - accuracy: 0.8060
Epoch 19: val_accuracy did not improve from 0.84695
61/61 [==============================] - 2s 26ms/step
 F1 macro : 0.7758882975177895
61/61 [==============================] - 6s 105ms/step - loss: 0.0080 - accuracy: 0.8060 - val_loss: 0.0205 - val_accuracy: 0.8165
Epoch 20/20
 3/61 [>.............................] - ETA: 3s - loss: 0.0062 - accuracy: 0.8438

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


61/61 [==============================] - ETA: 0s - loss: 0.0074 - accuracy: 0.8132
Epoch 20: val_accuracy did not improve from 0.84695
61/61 [==============================] - 2s 28ms/step
 F1 macro : 0.7805389343792973
61/61 [==============================] - 7s 111ms/step - loss: 0.0074 - accuracy: 0.8132 - val_loss: 0.0204 - val_accuracy: 0.7694


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [68]:
!gdown --id 107XgCALcMBKQSTLuLeWWRMpkUPOkkOV-

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=107XgCALcMBKQSTLuLeWWRMpkUPOkkOV-
To: /content/intent-detection-018--0.846953.h5
100% 438M/438M [00:02<00:00, 175MB/s]


In [69]:
intent_model.load_weights('intent-detection-018--0.846953.h5')

In [70]:
intent_model.evaluate([test_inp, test_mask], test_labels)

44/44 [==============================] - 1s 29ms/step - loss: 0.0204 - accuracy: 0.8463


[0.020415393635630608, 0.8463203310966492]

In [71]:
preds = intent_model.predict([test_inp, test_mask])
preds[preds>=0.5] = 1
preds[preds<0.5] = 0

44/44 [==============================] - 1s 28ms/step


In [72]:
print(classification_report(preds, test_labels))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       260
           1       0.83      0.89      0.86        28
           2       0.99      1.00      0.99       311
           3       1.00      0.98      0.99       124
           4       1.00      1.00      1.00       120
           5       0.93      0.74      0.83        35
           6       0.97      0.97      0.97        40
           7       0.98      0.97      0.98       537
           8       0.78      0.87      0.82        89
           9       1.00      1.00      1.00         5
          10       1.00      0.86      0.93        22
          11       0.75      1.00      0.86        18
          12       0.86      0.97      0.91        33
          13       0.91      0.86      0.89       123
          14       0.92      1.00      0.96        24
          15       1.00      1.00      1.00        28
          16       1.00      1.00      1.00        37
          17       0.00    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###b

In [73]:
!gdown --id 101uOTEhpm7t3Y4iIPpA_isGUHGKXafsA

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=101uOTEhpm7t3Y4iIPpA_isGUHGKXafsA
To: /content/intent-test.txt
100% 600/600 [00:00<00:00, 1.13MB/s]


In [74]:
def detect_intent(sentence):
  intents = []
  sent_inp, sent_mask = encode_text(bert_tokenizer, [sentence], max_length)
  predicted_intent = intent_model.predict([sent_inp, sent_mask])
  predicted_intent[predicted_intent>=0.5] = 1
  predicted_intent[predicted_intent<0.5] = 0
  for i in range(len(predicted_intent[0])):
    if predicted_intent[0][i] == 1:
      intents.append(intent_list[i])

  return intents

In [76]:
with open('intent-test.txt') as f:
  for sent in f:
    sent = sent.strip()
    intents = detect_intent(sent)
    print(sent)
    print(intents)
    print('\n\n')

1/1 [==============================] - 0s 44ms/step
is it in the cheap price range
['confirm_pricerange']



1/1 [==============================] - 0s 44ms/step
is there a cheap restaurant that serves vietnamese food
['confirm_pricerange', 'confirm_food']



1/1 [==============================] - 0s 42ms/step
can i please have their phone number
['request_phone']



1/1 [==============================] - 0s 43ms/step
no thank you good bye
['bye', 'negate']



1/1 [==============================] - 0s 43ms/step
i am looking for an expensive restaurant that serves australian food
['inform_pricerange', 'inform_food']



1/1 [==============================] - 0s 41ms/step
how about asian oriental food
['inform_food', 'reqalts']



1/1 [==============================] - 0s 46ms/step
i got to find a cheap restaurant that serves mexican food
['inform_pricerange', 'inform_food']



1/1 [==============================] - 0s 44ms/step
okay and uh whats their phone number please
['request_phone']

##Q3

###a

In [103]:
def get_all_restaurants(path):
  f = open (path, 'r')
  data = json.loads(f.read())
  db_results = []
  for i in range(len(data)):
    for res in data[i]:
      if 'db_result' in res:
        print('\n\n\n')
        if res['db_result']:
          db_results.append(res['db_result'])
  # f.close()
  return db_results

In [ ]:
restaurant_list = get_all_restaurants('simple-dstc2-trn.json')

In [105]:
# remove duplicates
restaurant_list = [dict(t) for t in {tuple(d.items()) for d in restaurant_list}]

In [106]:
restaurant_list = sorted(restaurant_list, key=lambda d: d['name']) 

###b

In [107]:
sorted_restaurant = []
keys_list = ['name', 'food', 'area', 'pricerange', 'phone', 'addr', 'postcode']
for i in range(len(restaurant_list)):
  items = sorted(restaurant_list[i].items(), key=lambda pair: keys_list.index(pair[0]))
  sorted_restaurant.append(items)

### c 

In [108]:
!gdown --id 1-snND1ySP8qslXLVScy6z7lSPZTc4WI4

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-snND1ySP8qslXLVScy6z7lSPZTc4WI4
To: /content/db-test.txt
100% 188/188 [00:00<00:00, 335kB/s]


In [109]:
db_test = []
with open('db-test.txt') as f:
  for query in f:
    d = json.loads('{'+query.strip(',\n')+'}')
    db_test.append(d)

In [110]:
def find_restaurant(query):
  res = []

  for key, value in query.items():
    tmp = []
    for restaurant in restaurant_list:
      if key in restaurant:
        if restaurant[key] == value:
          tmp.append(restaurant['name'])
    if len(res) == 0:
      res = tmp
    else:
      res = list(set(res) & set(tmp))  

  res.sort()
  return res

In [112]:
for query in db_test:
  result = find_restaurant(query)
  print(query)
  print(len(result))
  for value in result:
    print(value)
  print('\n'*5)

{'food': 'indian', 'phone': '01223 506055'}
1
tandoori palace






{'pricerange': 'cheap', 'area': 'east'}
1
the missing sock






{'phone': '01223 311911', 'area': 'centre'}
1
kymmoy






{'name': 'cocum'}
1
cocum






{'addr': '290 mill road city centre'}
1
royal standard








##Q4

In [113]:
templates = {}
with open('simple-dstc2-templates.txt') as f:
  for line in f:
    key, value = line.strip().split('\t')
    templates[key] = value

In [114]:
def get_addr(name):
  res = ''
  for rest in restaurant_list:
    if rest['name'] == name:
      res = rest['addr']
  return res

In [115]:
def get_phone(name):
  res = ''
  for rest in restaurant_list:
    if rest['name'] == name:
      res = rest['phone']
  return res

In [116]:
def get_postcode(name):
  res = ''
  for rest in restaurant_list:
    if rest['name'] == name:
      res = rest['postcode']
  return res

In [117]:
def get_res(cach_dict):
  tmp = cach_dict
  name = ''
  if 'food' in tmp and 'pricerange' in tmp and 'area' in tmp:
    res = find_restaurant(tmp)
    if len(res) > 0:
      sent = templates['inform_area+inform_food+inform_pricerange+offer_name']
      name = res[0]
      sent = sent.replace('#name', res[0])
      sent = sent.replace('#food', tmp['food'])
      sent = sent.replace('#pricerange', tmp['pricerange'])
      sent = sent.replace('#area', tmp['area'])
      print(sent)
    else:
      sent = templates['canthelp_area_food_pricerange']
      sent = sent.replace('#food', tmp['food'])
      sent = sent.replace('#pricerange', caches['pricerange'])
      sent = sent.replace('#area', tmp['area'])
      print(sent)
  elif 'food' in tmp and 'pricerange' in tmp and 'area' not in tmp:
    res = find_restaurant(tmp)
    if len(res) > 0:
      name = res[0]
      sent = templates['inform_food+inform_pricerange+offer_name']
      sent = sent.replace('#name', res[0])
      sent = sent.replace('#food', tmp['food'])
      sent = sent.replace('#pricerange', tmp['pricerange'])
      print(sent)
    else:
      sent = templates['canthelp_food_pricerange']
      sent = sent.replace('#food', tmp['food'])
      sent = sent.replace('#pricerange', tmp['pricerange'])
      print(sent)
  elif 'food' in tmp and 'area' in tmp and 'pricerange' not in tmp:
    res = find_restaurant(tmp)
    if len(res) > 0:
      name = res[0]
      sent = templates['inform_area+inform_food+offer_name']
      sent = sent.replace('#name', res[0])
      sent = sent.replace('#food', tmp['food'])
      sent = sent.replace('#area', tmp['area'])
      print(sent)
    else:
      sent = templates['canthelp_area_food']
      sent = sent.replace('#food', tmp['food'])
      sent = sent.replace('#area', tmp['area'])
      print(sent) 
  elif 'food' in tmp and 'pricerange' not in tmp and 'area' not in tmp:
    res = find_restaurant(tmp)
    if len(res) > 0:
      name = res[0]
      sent = templates['inform_food+offer_name']
      sent = sent.replace('#name', res[0])
      sent = sent.replace('#food', tmp['food'])
      print(sent)
    else:
      sent = templates['canthelp_food_pricerange']
      sent = sent.replace('#food', tmp['food'])
      print(sent)
  elif 'food' not in tmp and 'pricerange' in tmp and 'area' in tmp:
    res = find_restaurant(tmp)
    if len(res) > 0:
      name = res[0]
      sent = templates['inform_area+inform_pricerange+offer_name']
      sent = sent.replace('#name', res[0])
      sent = sent.replace('#pricerange', tmp['pricerange'])
      sent = sent.replace('#area', tmp['area'])
      print(sent)
    else:
      sent = templates['canthelp_area_pricerange']
      sent = sent.replace('#pricerange', tmp['pricerange'])
      sent = sent.replace('#area', tmp['area'])
      print(sent)
  elif 'food' not in tmp and 'pricerange' in tmp and 'area' not in tmp:
    res = find_restaurant(tmp)
    if len(res) > 0:
      name = res[0]
      sent = templates['inform_pricerange+offer_name']
      sent = sent.replace('#name', res[0])
      sent = sent.replace('#pricerange', tmp['pricerange'])
      print(sent)
    else:
      sent = templates['canthelp_pricerange']
      sent = sent.replace('#pricerange', tmp['pricerange'])
      print(sent)
  elif 'food' not in tmp and 'pricerange' not in tmp and 'area' in tmp:
    res = find_restaurant(tmp)
    if len(res) > 0:
      name = res[0]
      sent = templates['inform_area+offer_name']
      sent = sent.replace('#name', res[0])
      sent = sent.replace('#area', tmp['area'])
      print(sent)
    else:
      sent = templates['canthelp_area']
      sent = sent.replace('#area', tmp['area'])
      print(sent)
  return name

In [127]:
def process_query(sentence, caches, rest_name):

  intents = detect_intent(sentence)
  slots, tags = fill_slots(sentence)
  name = ''
  if len(intents) == 0:
    print(templates['confirm-domain'])
  elif 'hello' in intents:
    print(templates['welcomemsg'])
  elif 'thankyou' in intents:
    print(templates['bye'])
  elif 'bye' in intents:
    print('Bye, Have a Great Time!')
    caches = {'pricerange':'', 'food':'', 'area':''}
    name = ''
  elif 'restart' in intents:
    print('restart system')
    caches = {'pricerange':'', 'food':'', 'area':''}
    name = ''
  elif len(intents) == 1:
    if intents[0] == 'inform_pricerange':
      if 'pricerange' in slots:
        pricerange = slots['pricerange']
        caches['pricerange'] = pricerange
        tmp = {key:val for key, val in caches.items() if val != ''}
        name = get_res(tmp)
        if caches['food'] == '':
          print(templates['request_food'])
        
    elif intents[0] == 'inform_food':
      if 'food' in slots:
        food = slots['food']
        caches['food'] = food
        tmp = {key:val for key, val in caches.items() if val != ''}
        name = get_res(tmp)
    elif intents[0] == 'inform_area':
      if 'area' in slots:
        area = slots['area']
        caches['area'] == area
        tmp = {key:val for key, val in caches.items() if val != ''}
        name = get_res(tmp)        
        if caches['food'] == '':
          print(templates['request_food'])

    elif intents[0] == 'request_addr':
      sent = templates['inform_addr+offer_name']
      sent = sent.replace('#name', rest_name)
      sent = sent.replace('#addr', get_addr(rest_name))
      print(sent)
    elif intents[0] == 'request_phone':
      sent = templates['inform_phone+offer_name']
      sent = sent.replace('#name', rest_name)
      sent = sent.replace('#phone', get_phone(rest_name))
      print(sent)

    else:
      print('I dont know')
  
  elif len(intents) == 2:
    if 'inform_food' in intents and 'inform_pricerange' in intents:
      pricerange = slots['pricerange']
      food = slots['food']
      caches['pricerange'] = pricerange
      caches['food'] = food
      tmp = {key:val for key, val in caches.items() if val != ''}
      name = get_res(tmp)
    elif 'inform_food' in intents and 'inform_area' in intents:
      area = slots['area']
      food = slots['food']
      caches['food'] = food
      caches['area'] = area
      tmp = {key:val for key, val in caches.items() if val != ''}
      name = get_res(tmp)

    elif 'inform_pricerange' in intents and 'inform_area' in intents:
      area = slots['area']
      pricerange = slots['pricerange']
      caches['area'] = area
      caches['pricerange'] = pricerange
      tmp = {key:val for key, val in caches.items() if val != ''}
      name = get_res(tmp)  
    elif 'inform_area' in intents:
      area = slots['area']
      caches['area'] = area
      tmp = {key:val for key, val in caches.items() if val != ''}
      name = get_res(tmp)  
    else:
      print('I dont know')      

  elif len(intents) == 3:
    if 'inform_food' in intents and 'inform_pricerange' in intents and 'inform_area' in intents:
      area = slots['area']
      pricerange = slots['pricerange']
      food = slots['food']
      caches['food'] = food
      caches['pricerange'] = pricerange
      caches['area'] = area
      name = get_res(caches)
  
  else:
    print('I dont know')
  return caches,name

In [129]:
caches = {'pricerange':'', 'food':'', 'area':''}
rest_name = ''
while(True):
  sent = input()
  if sent == 'exit':
    break
  caches, rest_name = process_query(sent, caches, rest_name)

hi
1/1 [==============================] - 0s 44ms/step
Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
a cheap restaurant plz
1/1 [==============================] - 0s 44ms/step
The price range at ask is cheap.
What kind of food would you like?
chinese
1/1 [==============================] - 0s 40ms/step
charlie chan serves chinese food in the cheap price range.
address
1/1 [==============================] - 0s 47ms/step
Sure, charlie chan is on regent street city centre.
what about west area?
1/1 [==============================] - 0s 42ms/step
Sorry there is no cheap restaurant in the west of town serving chinese food.
thanks
1/1 [==============================] - 0s 42ms/step
You are welcome!
bye
1/1 [==============================] - 0s 42ms/step
Bye, Have a Great Time!
exit


In [130]:
caches = {'pricerange':'', 'food':'', 'area':''}
rest_name = ''

In [131]:
detect_intent('cheap restaurant')

1/1 [==============================] - 0s 46ms/step


['inform_pricerange']